In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.population import WorldpopPopulation, FacebookPopulation
from pisa.isopolygons import OsmIsopolygonCalculatorAlternative


### Define administrative area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm)! 

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [3]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

INFO:pisa.administrative_area:Validating country name: Timor-Leste
INFO:pisa.administrative_area:Country name 'Timor-Leste' validated successfully
INFO:pisa.administrative_area:Retrieving boundaries of all administrative areas of level 1 for country Timor-Leste


### Get hospitals from osm

In [4]:
hospitals = Facilities(admin_area_boundaries=baucau).get_existing_facilities()

# todo: suppress user warning Geometry is in a geographic CRS, and
# add documentation on why this is ok

INFO:pisa.facilities:Retrieving existing facilities with tags {'amenity': 'hospital', 'building': 'hospital'} using OSM.
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/facilities.py:63: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  facilities_gdf["longitude"] = facilities_gdf.geometry.centroid.x
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/facilities.py:64: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  facilities_gdf["latitude"] = facilities_gdf.geometry.centroid.y
INFO:pisa.facilities:Successfully retrieved existing facilities from OSM.


In [6]:
hospitals.head()

,longitude,latitude
osmid,,
1777614876,126.600478,-8.547331
1777614896,126.653327,-8.628429
1777615024,126.596990,-8.474279
1777615035,126.726988,-8.656430
1777615036,126.684393,-8.567978


### Get population from WorldPop

In [ ]:
# TODO: why "ID" column? extra index for what reason? Maybe becomes apparent later?

population = WorldpopPopulation(
    admin_area_boundaries=baucau,
    iso3_country_code=timor_leste.get_iso3_country_code()                
).get_population_gdf()

population.head()

,ID,longitude,latitude,population,geometry
0,0,126.14917,-8.67333,0.10,POINT (126.14917 -8.67333)
1,1,126.14917,-8.67250,0.09,POINT (126.14917 -8.6725)
2,2,126.15000,-8.67333,0.10,POINT (126.15 -8.67333)
3,3,126.15000,-8.67250,0.09,POINT (126.15 -8.6725)
4,4,126.15000,-8.67167,0.09,POINT (126.15 -8.67167)


### Alternatively: get population from facebook

In [8]:
population_fb = FacebookPopulation(
    admin_area_boundaries=baucau,
    iso3_country_code=timor_leste.get_iso3_country_code()                
).get_population_gdf()

population_fb.head()

INFO:hdx.api.configuration:No HDX base configuration parameter. Using default base configuration file: /Users/catherine/Library/Caches/pypoetry/virtualenvs/gpbp-ys_3FNdB-py3.10/lib/python3.10/site-packages/hdx/api/hdx_base_configuration.yml.
INFO:hdx.api.configuration:Loading HDX base configuration from: /Users/catherine/Library/Caches/pypoetry/virtualenvs/gpbp-ys_3FNdB-py3.10/lib/python3.10/site-packages/hdx/api/hdx_base_configuration.yml


INFO:hdx.api.configuration:No HDX configuration parameter and no configuration file at default path: /Users/catherine/.hdx_configuration.yml.
INFO:hdx.api.configuration:Read only access to HDX: True


,ID,longitude,latitude,population,geometry
0,0,126.17236,-8.50986,5.44,POINT (126.17236 -8.50986)
1,1,126.17681,-8.50625,5.44,POINT (126.17681 -8.50625)
2,2,126.17736,-8.51042,5.44,POINT (126.17736 -8.51042)
3,3,126.18069,-8.51208,5.44,POINT (126.18069 -8.51208)
4,4,126.18097,-8.51264,5.44,POINT (126.18097 -8.51264)


### Isopolygons

Here we make some choices:
- mode of transport 
- distance type

Valid values are in the script pisa.constants

#### Option 1: using OSM

For this, we need to get the road network from osmnx

In [ ]:
from pisa.osm_road_network import OsmRoadNetwork

# suppose the roads here are difficult: the user should be able to adjust the fallback speed to a lower value

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau,
    mode_of_transport="driving",
    distance_type="travel_time",
    fallback_speed=30
).get_osm_road_network()